In [1]:
import pandas as pd
import ast
from collections import Counter

In [2]:
df = pd.read_parquet(r"C:\Users\user\Desktop\SKN_AFTER_STUDY\data\0000.parquet", engine="pyarrow")
df['annotation'] = df['annotation'].apply(ast.literal_eval)

In [3]:
def prepro_annotation(labels:list):
    dic_list = [dic['emotion'] for dic in labels]
    return dic_list

def majority_first_value(pairs, min_count=3):
    if not pairs:  # None, 빈 리스트 처리
        return None
    first_values = [pair[0] for pair in pairs]
    counts = Counter(first_values)
    for val, cnt in counts.items():
        if cnt >= min_count:
            return val
    return None

def majority_second_value(pairs, min_count=3):
    if not pairs:
        return None
    second_values = [pair[1] for pair in pairs]
    counts = Counter(second_values)
    for val, cnt in counts.items():
        if cnt >= min_count:
            return val
    return None

# def has_majority(labels: list, min_count=3):
#     counts = Counter(labels)
#     return any(cnt >= min_count for cnt in counts.values())

def max_value(lables:list):
    counts = Counter(lables)
    return counts.most_common(1)[0][0]

def has_majority(labels: list, index: int, min_count=3):
    values = [label[index] for label in labels]
    counts = Counter(values)
    most_frequent = counts.most_common(1)[0][0] if counts else None
    return any(cnt >= min_count for cnt in counts.values()), most_frequent

# 각 행에 대해 label1과 label2 생성
def get_labels(annotations_split):
    has_maj_0, most_freq_0 = has_majority(annotations_split, 0)
    has_maj_1, most_freq_1 = has_majority(annotations_split, 1)

    label1 = most_freq_0 if has_maj_0 else None
    label2 = most_freq_1 if has_maj_1 else None

    return label1, label2

def get_label2(label_list, index):
    a,b = label_list[index]
    return a,b

### 데이터 중분류, 소분류 확인

In [4]:
df['annotation']= df['annotation'].apply(prepro_annotation)

In [5]:
df["annotations_split"] = df["annotation"].apply(lambda x: [item.split("_") for item in x])

In [6]:
annotations_list = [pair for sublist in df['annotations_split'] for pair in sublist]

In [7]:
label1 = [label[0] for label in annotations_list]
label2 = [label[1] for label in annotations_list]

In [8]:
print(set(label1))

{'중립', '두려움', '수치심', '기쁨', '욕망', '미움(상대방)', '분노', '싫어함(상태)', '사랑', '슬픔'}


In [9]:
print(set(label2))

{'공포', '신뢰감', '반감', '두근거림', '궁금함', '다정함', '불신감', '냉담', '안정감', '날카로움', '자신감', '사나움', '경멸', '감동', '허망', '기대감', '초조함', '그리움', '싫증', '시기심', '비위상함', '타오름', '무기력', '욕심', '놀람', '불만', '매력적', '반가움', '미안함', '동정(슬픔)', '아른거림', '갈등', '고통', '수치심', '후회', '답답함', '발열', '위축감', '아픔', '통쾌함', '난처함', '열정적인', '즐거움', '걱정', '부끄러움', '아쉬움', '심심함', '신명남', '원망', '호감', '억울함', '죄책감', '외로움', '불편함', '서먹함', '불쾌', '편안함', '치사함', '자랑스러움', '실망', '너그러움', '공감', '외면', '절망', '귀중함', '만족감', '고마움'}


In [10]:
print(len(set(label1)))
print(len(set(label2)))

10
67


In [11]:
df['label0_0'] = ''
df['label0_1'] = ''
df['label1_0'] = ''
df['label1_1'] = ''
df['label2_0'] = ''
df['label2_1'] = ''
df['label3_0'] = ''
df['label3_1'] = ''
df['label4_0'] = ''
df['label4_1'] = ''
for idx, labels in enumerate(df['annotations_split']):
  a,b = get_label2(labels, 0)
  df.loc[idx,'label0_0'] = a
  df.loc[idx,'label0_1'] = b

for idx, labels in enumerate(df['annotations_split']):
  a,b = get_label2(labels, 1)
  df.loc[idx,'label1_0'] = a
  df.loc[idx,'label1_1'] = b

for idx, labels in enumerate(df['annotations_split']):
  a,b = get_label2(labels, 2)
  df.loc[idx,'label2_0'] = a
  df.loc[idx,'label2_1'] = b

for idx, labels in enumerate(df['annotations_split']):
  a,b = get_label2(labels, 3)
  df.loc[idx,'label3_0'] = a
  df.loc[idx,'label3_1'] = b

for idx, labels in enumerate(df['annotations_split']):
  a,b = get_label2(labels, 4)
  df.loc[idx,'label4_0'] = a
  df.loc[idx,'label4_1'] = b

In [ ]:
all_values = [
    # 기쁨
    "반가움", "즐거움", "신명남", "자신감", "감동", "만족감", "편안함", "고마움", "신뢰감", "안정감", "공감", "자랑스러움",
    # 사랑
    "호감", "귀중함", "매력적", "두근거림", "아른거림", "너그러움", "열정적인", "다정함", "동정(슬픔)",
    # 슬픔
    "억울함", "외로움", "후회", "실망", "허망", "그리움", "수치심", "고통", "절망", "무기력", "아픔",
    # 두려움
    "위축감", "놀람", "공포", "걱정", "초조함",
    # 분노
    "원망", "불쾌", "날카로움", "타오름",
    # 미움(상대방)
    "반감", "경멸", "비위상함", "치사함", "불신감", "시기심", "외면", "냉담",
    # 욕망
    "욕심", "궁금함", "아쉬움", "불만", "갈등", "기대감",
    # 싫어함(상태)
    "답답함", "불편함", "난처함", "서먹함", "심심함", "싫증",
    # 수치심
    "부끄러움", "죄책감", "미안함"
]
len(all_values)

# 기쁨
joy_set = {
    "반가움", "즐거움", "신명남", "자신감", "감동", "만족감",
    "편안함", "고마움", "신뢰감", "안정감", "공감", "자랑스러움"
}

# 사랑
love_set = {
    "호감", "귀중함", "매력적", "두근거림", "아른거림",
    "너그러움", "열정적인", "다정함", "동정(슬픔)"
}

# 슬픔
sadness_set = {
    "억울함", "외로움", "후회", "실망", "허망", "그리움",
    "수치심", "고통", "절망", "무기력", "아픔"
}

# 두려움
fear_set = {
    "위축감", "놀람", "공포", "걱정", "초조함"
}

# 분노
anger_set = {
    "원망", "불쾌", "날카로움", "타오름"
}

# 미움(상대방)
hate_set = {
    "반감", "경멸", "비위상함", "치사함", "불신감",
    "시기심", "외면", "냉담"
}

# 욕망
desire_set = {
    "욕심", "궁금함", "아쉬움", "불만", "갈등", "기대감"
}

# 싫어함(상태)
dislike_set = {
    "답답함", "불편함", "난처함", "서먹함", "심심함", "싫증"
}

# 수치심
shame_set = {
    "부끄러움", "죄책감", "미안함"
}

In [13]:
all_labels = set(label2)
all_values = set(all_values)
other_values = all_labels - all_values
print(other_values)

{'발열', '사나움', '통쾌함'}


In [14]:
label1_value = set(label1)
print(label1_value)
label1_value = list(label1_value)

{'중립', '두려움', '수치심', '기쁨', '욕망', '미움(상대방)', '분노', '싫어함(상태)', '사랑', '슬픔'}


In [15]:
# 로우를 기준을 25개 이상인 경우
for label1 in label1_value:
  for value in other_values:
    df_filtered_exclude = df[
      ((df['label0_0'] == label1) & (df['label0_1']==value)) |
      ((df['label1_0'] == label1) & (df['label1_1']==value)) |
      ((df['label2_0'] == label1) & (df['label2_1']==value)) |
      ((df['label3_0'] == label1) & (df['label3_1']==value)) |
      ((df['label4_0'] == label1) & (df['label4_1']==value))
  ]
    if len(df_filtered_exclude) >=25:
      print(f'{label1}의 {value} 갯수 :',len(df_filtered_exclude))

분노의 발열 갯수 : 27
분노의 사나움 갯수 : 33


In [59]:
all_values = set(label2)

# 기쁨
joy_set = {
    "반가움", "즐거움", "신명남", "자신감", "감동", "만족감",
    "편안함", "고마움", "신뢰감", "안정감", "공감", "자랑스러움"
}
not_joy_values = all_values - joy_set

# 사랑
love_set = {
    "호감", "귀중함", "매력적", "두근거림", "아른거림",
    "너그러움", "열정적인", "다정함", "동정(슬픔)"
}
not_love_values = all_values - love_set

# 슬픔
sadness_set = {
    "억울함", "외로움", "후회", "실망", "허망", "그리움",
    "수치심", "고통", "절망", "무기력", "아픔"
}
not_sadness_values = all_values - sadness_set

# 두려움
fear_set = {
    "위축감", "놀람", "공포", "걱정", "초조함"
}
not_fear_values = all_values - fear_set

# 분노
anger_set = {
    "원망", "불쾌", "날카로움", "타오름"
}
not_anger_values = all_values - anger_set

# 미움(상대방)
hate_set = {
    "반감", "경멸", "비위상함", "치사함", "불신감",
    "시기심", "외면", "냉담"
}
not_hate_values = all_values - hate_set

# 욕망
desire_set = {
    "욕심", "궁금함", "아쉬움", "불만", "갈등", "기대감"
}
not_desire_values = all_values - desire_set

# 싫어함(상태)
dislike_set = {
    "답답함", "불편함", "난처함", "서먹함", "심심함", "싫증"
}
not_dislike_values = all_values - dislike_set

# 수치심
shame_set = {
    "부끄러움", "죄책감", "미안함"
}
not_shame_values = all_values - shame_set

In [ ]:
not_values_list = [
    "not_joy_values",
    "not_love_values",
    "not_sadness_values",
    "not_fear_values",
    "not_anger_values",
    "not_hate_values",
    "not_desire_values",
    "not_dislike_values",
    "not_shame_values",
]

In [17]:
# 라벨별 기준 25개 이상인 경우
label_cols = [
    ('label0_0', 'label0_1'),
    ('label1_0', 'label1_1'),
    ('label2_0', 'label2_1'),
    ('label3_0', 'label3_1'),
    ('label4_0', 'label4_1'),
]

# 초기화
counts = {label1: {value: 0 for value in other_values} for label1 in label1_value}

In [18]:
for label1 in label1_value:
    for value in other_values:
        total = 0
        for col0, col1 in label_cols:
            total += len(df[(df[col0] == label1) & (df[col1] == value)])
        counts[label1][value] = total

In [196]:
for label1 in counts:
    for value in counts[label1]:
        print(f'{label1}의 {value} 갯수 : {counts[label1][value]}')

미움(상대방)의 통쾌함 갯수 : 0
미움(상대방)의 사나움 갯수 : 0
미움(상대방)의 발열 갯수 : 0
두려움의 통쾌함 갯수 : 0
두려움의 사나움 갯수 : 0
두려움의 발열 갯수 : 0
사랑의 통쾌함 갯수 : 0
사랑의 사나움 갯수 : 0
사랑의 발열 갯수 : 0
슬픔의 통쾌함 갯수 : 0
슬픔의 사나움 갯수 : 0
슬픔의 발열 갯수 : 0
중립의 통쾌함 갯수 : 1
중립의 사나움 갯수 : 0
중립의 발열 갯수 : 0
분노의 통쾌함 갯수 : 0
분노의 사나움 갯수 : 44
분노의 발열 갯수 : 38
기쁨의 통쾌함 갯수 : 27
기쁨의 사나움 갯수 : 0
기쁨의 발열 갯수 : 0
싫어함(상태)의 통쾌함 갯수 : 0
싫어함(상태)의 사나움 갯수 : 0
싫어함(상태)의 발열 갯수 : 0
수치심의 통쾌함 갯수 : 0
수치심의 사나움 갯수 : 0
수치심의 발열 갯수 : 0
욕망의 통쾌함 갯수 : 0
욕망의 사나움 갯수 : 0
욕망의 발열 갯수 : 0


In [ ]:
# 기쁨
joy_set = {
    "반가움", "즐거움", "신명남", "자신감", "감동", "만족감",
    "편안함", "고마움", "신뢰감", "안정감", "공감", "자랑스러움"
}

all_values = set(label2)

not_joy_values = all_values - joy_set
print(len(not_joy_values))

55


In [ ]:
label_pairs = [
    ('label0_0', 'label0_1'),
    ('label1_0', 'label1_1'),
    ('label2_0', 'label2_1'),
    ('label3_0', 'label3_1'),
    ('label4_0', 'label4_1'),
]

for value in not_joy_values:
    total = 0
    for col0, col1 in label_pairs:
        total += len(df[(df[col0] == '기쁨') & (df[col1] == value)])

    if total >= 25:
        print(f'기쁨의 {value} 갯수 :', total)

In [ ]:
# 중분류별 소분류 중에서 25개 이상인 것만 출력
category_sets = {
    '기쁨': {
        "반가움", "즐거움", "신명남", "자신감", "감동", "만족감",
        "편안함", "고마움", "신뢰감", "안정감", "공감", "자랑스러움"
    },
    '사랑': {
        "호감", "귀중함", "매력적", "두근거림", "아른거림",
        "너그러움", "열정적인", "다정함", "동정(슬픔)"
    },
    '슬픔': {
        "억울함", "외로움", "후회", "실망", "허망", "그리움",
        "수치심", "고통", "절망", "무기력", "아픔"
    },
    '두려움': {
        "위축감", "놀람", "공포", "걱정", "초조함"
    },
    '분노': {
        "원망", "불쾌", "날카로움", "타오름"
    },
    '미움(상대방)': {
        "반감", "경멸", "비위상함", "치사함", "불신감",
        "시기심", "외면", "냉담"
    },
    '욕망': {
        "욕심", "궁금함", "아쉬움", "불만", "갈등", "기대감"
    },
    '싫어함(상태)': {
        "답답함", "불편함", "난처함", "서먹함", "심심함", "싫증"
    },
    '수치심': {
        "부끄러움", "죄책감", "미안함"
    }
}

all_values = set(label2)

label_pairs = [
    ('label0_0', 'label0_1'),
    ('label1_0', 'label1_1'),
    ('label2_0', 'label2_1'),
    ('label3_0', 'label3_1'),
    ('label4_0', 'label4_1'),
]

for category, belongs_to_values in category_sets.items():
    print(f"\n=== {category} ===")
    not_belongs_values = all_values - belongs_to_values
    
    for value in not_belongs_values:
        total = 0
        for col0, col1 in label_pairs:
            total += len(df[(df[col0] == category) & (df[col1] == value)])
        
        if total >= 25:
            print(f'{category}의 {value} 갯수: {total}')

# 중립 처리 (중립은 별도 세트가 없으므로)
print(f"\n=== 중립 ===")
for value in all_values:
    total = 0
    for col0, col1 in label_pairs:
        total += len(df[(df[col0] == '중립') & (df[col1] == value)])
    
    if total >= 25:
        print(f'중립의 {value} 갯수: {total}')


=== 기쁨 ===
기쁨의 통쾌함 갯수: 27
기쁨의 기대감 갯수: 25
기쁨의 놀람 갯수: 26

=== 사랑 ===

=== 슬픔 ===
슬픔의 동정(슬픔) 갯수: 89

=== 두려움 ===

=== 분노 ===
분노의 발열 갯수: 38
분노의 사나움 갯수: 44

=== 미움(상대방) ===

=== 욕망 ===

=== 싫어함(상태) ===

=== 수치심 ===

=== 중립 ===
중립의 놀람 갯수: 31
